In [38]:
import json
import hashlib
import base64
import pandas as pd
import math
import time
import requests
import datetime

In [39]:
appId = 'xm_india'
appKey = 'QoTyhvmuSBCUlkYFfRGw0JZNzidWMgXsKpD6VHE3j72A8t1Lreb5xOP9nq4Iac'

In [40]:
cnData = pd.read_excel('../InputData/InputUpload.xlsx')

In [41]:
cnData["API_Response"]=cnData["API_Response"].values.astype('str')

In [42]:
for index in range(len(cnData)):
    if (math.isnan(cnData.at[index,'SAP_Code']) == False and math.isnan(cnData.at[index,'CN_Number']) == False):
        sap_code = int(cnData.at[index,'SAP_Code'])
        cn_no = str("0" + str(cnData.at[index,'CN_Number']))
        pos_date = cnData.at[index,'Posting_Date']
        pos_date_str = pos_date.strftime('%Y-%m-%d')
        pos_date_dt = datetime.datetime.strptime(pos_date_str, "%Y-%m-%d")
        ins_date = time.mktime(pos_date_dt.timetuple())
        file_link = cnData.at[index,'URL']
        #Scheme_Ref_No = cnData.at[index,"Scheme_Ref_No"]
        body = {
                "sap_code": sap_code,
                "cn_no": cn_no,
                "ins_date": ins_date,
                "file_link": file_link,
                # "scheme_ref_no": Scheme_Ref_No
        }
        # print(body)
        body = json.dumps(body)
        #print((appId + body + appKey).encode())
        sign = hashlib.md5((appId + body + appKey).encode())
        sign = sign.hexdigest()
        sign = sign.upper()
        # print(sign)
        data = {
            'header':{
                'appid':appId,
                'sign':sign
            },
            'body':body
        }
        base64_bytes = base64.b64encode(json.dumps(data).encode())
        #print(base64_bytes)
        
        # api-endpoint
        URL = "https://in-dms.pre.mi.com/api/cn-copy/create-cn/"
        
        # defining a params dict for the parameters to be sent to the API
        data = base64_bytes
        
        # sending post request and saving response as response object
        r = requests.post(url = URL, data = data)

        # extracting response text
        response = r.json()
        cnData.at[index,'API_Response'] = str(response)
        
       
        
        


In [43]:
cnData.to_excel('API_Response.xlsx')